In [8]:
# 匯入模組
import numpy as np
import pandas as pd
from math import sqrt
from scipy.stats import skewnorm, norm
import time
import random

In [2]:
# 門檻 和 債務人個數
cc1 = 200
cc2 = 800
# 債務人個數
m = cc1+cc2

# 系統風險因子的因子負荷量
ak1 = 0.8
bk1 = sqrt(1-(ak1)**2)
ak2 = 0.5
bk2 = sqrt(1-(ak2)**2)

# 債務人違約時所產生的損失
ck1 = 2
ck2 = 1

# 個別債務人的違約機率
pk1 = 0.01
pk2 = 0.05

# SN 模型參數轉換
lk1 = ak1/sqrt(2-(ak1)**2)
lk2 = ak2/sqrt(2-(ak2)**2)

# CSN分配在 1-p 之值，對應回去的Z值
xk1 = skewnorm.ppf(1-pk1, a = lk1, loc = 0, scale = 1)
xk2 = skewnorm.ppf(1-pk2, a = lk2, loc = 0, scale = 1)

# 模擬次數
N = 1000000
NN = 1

# 數列
X = np.arange(10, 1200, 10)
I = np.zeros((N, len(X)))
L = np.zeros((N, NN))
Com = np.zeros((N,2))

In [3]:
xk1,xk2

(2.5667387939380486, 1.8630862356130053)

In [4]:
# 初始時間
start = time.time()

# 模擬過程
for j in range(N) :
    z = skewnorm.rvs(1, size = 1)
    ek1 = skewnorm.rvs(1, size = cc1)
    ek2 = skewnorm.rvs(1, size = cc2)
    XK1 = ak1*z + bk1*ek1
    XK2 = ak2*z + bk2*ek2
    Com[j,0]= np.sum((XK1>xk1)*1)
    Com[j,1]= np.sum((XK2>xk2)*1)
    L[j] = np.sum((XK1>xk1)*2) + np.sum((XK2>xk2)*1)
    for k in range(len(X)):
        I[j, k] = np.sum(L[j]>X[k])/NN
    
prob = np.zeros((len(X), NN))
for i in range(len(X)):
    prob[i] = sum(I[:, i])/N

# 結束時間
end = time.time()
spend = end - start
print(f'共花費 {spend:.2f} 秒')

共花費 1091.86 秒


In [5]:
var = np.zeros((len(X),1)) 
for i in range(len(X)):
    var[i] = prob[i]*(1-prob[i])
prob = pd.DataFrame(prob)
var = pd.DataFrame(var)
X = pd.DataFrame(X)
DATA = pd.concat([X, prob,var], axis = 1)
DATA.columns = ['風險值', '估計值','變異數']
DATA.to_csv('MC_DEP_CSN_1000_B1_0609.csv', index = False, encoding = 'utf_8_sig')
DATA

,風險值,估計值,變異數
0,10,0.911876,0.080358
1,20,0.796221,0.162253
2,30,0.688138,0.214604
3,40,0.593814,0.241199
4,50,0.513483,0.249818
5,60,0.444545,0.246925
6,70,0.386636,0.237149
7,80,0.336926,0.223407
8,90,0.294909,0.207938
9,100,0.259124,0.191979


In [6]:
# 查看 總違約機率分別為(0.05 ,0.01 ,0.005)時，所對應之風險值
x_1 = np.where(prob <= 0.05)[0][0] 
x_2 = np.where(prob <= 0.01)[0][0] 
x_3 = np.where(prob <= 0.005)[0][0] 
print(x_1,x_2,x_3)

26 49 60


In [13]:
prob.iloc[x_1][0]

0.047481

In [14]:
# 找出 0.05 機率對應的風險值
# 風險值 = 
print(f'風險值為 {(x_1+1)*10} 的估計值為 {prob.iloc[x_1][0]} ，變異數為 {var.iloc[x_1][0]}')

# 找出機率 0.01 對應的風險值
# 風險值 = 
print(f'風險值為 {(x_2+1)*10} 的估計值為 {prob.iloc[x_2][0]} ，變異數為 {var.iloc[x_2][0]}')

# 找出機率 0.005 對應的風險值
# 風險值 = 
print(f'風險值為 {(x_3+1)*10} 的估計值為 {prob.iloc[x_3][0]} ，變異數為 {var.iloc[x_3][0]}')

風險值為 270 的估計值為 0.047481 ，變異數為 0.045226554639
風險值為 500 的估計值為 0.009892 ，變異數為 0.009794148336
風險值為 610 的估計值為 0.004935 ，變異數為 0.004910645775
